# Install the necessary Python Packages

In [3]:
! pip install openai
! pip install pydub
! pip install fpdf
! pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.2/220.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40703 sha256=b7c52511bb5de67e62163642748f6278d0e055c482769985b150c7ae4dac2c43
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.9 MB/s

# Import Libraries

In [4]:
from google.colab import userdata
import requests
import base64
from openai import OpenAI
import io
from pydub import AudioSegment
from pydub.playback import play
from IPython.display import Audio, display
import PyPDF2

#
client = OpenAI(api_key="INSERT YOUR OPENAI_API_KEY HERE")

# Chat Completions

In [6]:
def stream_gpt(query, model):

    completion = client.chat.completions.create(
        model=model,
        messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": query}
        ],
        stream=True
    )

    for chunk in completion:
        print(chunk.choices[0].delta.content, end='')



In [7]:
stream_gpt(query="Tell me about St. John's University", model="gpt-3.5-turbo")

St. John's University is a private Catholic university located in New York City. It was founded in 1870 and is one of the largest Catholic universities in the United States. Known for its diverse community, St. John's has campuses in Queens, Staten Island, Manhattan, and Rome, Italy.

The university offers a wide range of academic programs, including over 100 undergraduate majors and minors, as well as various graduate and professional programs across disciplines such as business, law, education, pharmacy, and the sciences.

St. John's University is committed to providing a well-rounded education that incorporates the values of the Catholic and Vincentian traditions. It focuses on developing the whole person by encouraging intellectual, spiritual, social, and ethical growth. The university also encourages students to engage in service and social justice activities, reflecting its commitment to the Vincentian mission.

St. John's has a vibrant campus life with numerous student organizat

In [8]:
stream_gpt(query="Tell me about St. John's University", model="gpt-4")

St. John's University is a private, Roman Catholic, research university in New York City, United States. Founded by the Congregation of the Mission (C.M., the Vincentian Fathers) in 1870, the school was originally located in the neighborhood of Bedford–Stuyvesant in the borough of Brooklyn. In the 1950s, the school was relocated to its current site at Utopia Parkway in Hillcrest, Queens. St. John's also has campuses in Staten Island, Manhattan, and overseas in Rome, Italy.

It offers more than 100 bachelor, master, and doctoral degree programs in six colleges and schools. The university is noted for its programs in Business (The Peter J. Tobin College of Business), Education (The School of Education), Pharmacy (The College of Pharmacy and Health Sciences), and Law (School of Law).

The university is recognized for its diverse student population, representing 46 U.S states and 127 foreign countries. 

The school mascot is Johnny the Thunderbird and its sports teams participate in the NC

# GPT-4 Vision

In [9]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [11]:
def stream_gpt_vision(query, image_path):

    base64_image = encode_image(image_path)
    image_input = [
        {
            "type": "text",
            "text": query,
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
            },
        }
    ]

    completion = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages = [
            {"role": "system", "content": "You are an expert at turning images of mock-up designs into full fledged code. Give me the HTML and CSS code to create a website like the image attached."},
            {"role": "user", "content": image_input}
        ],
        max_tokens=500
    )

    print(completion.choices[0].message.content)
    return completion






In [12]:
res = stream_gpt_vision(query="Give me the HTML and CSS code for the image attached", image_path="/content/landing_page_example.png")

Certainly! Based on the design provided in the image, I'll outline the HTML and CSS structure to recreate a similar looking layout. For the shape around the "Hero Image" text, I'll use an abstract CSS shape through `clip-path`. This is just a basic outline and would need to be expanded upon for full interactivity and responsivity.

Here's an approximate code representation:

**HTML:**
```html
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Mockup Design</title>
<link rel="stylesheet" href="styles.css">
</head>
<body>
    <div class="container">
        <header>
            <div class="menu">☰</div>
            <button class="contact">Contact</button>
        </header>
        <main>
            <section class="hero">
                <div class="headline">
                    <h1>A Thought-Provoking Headline</h1>
                    <p>A quick value proposition that will make your visitors want 

# Text to Speech

In [7]:
def stream_text_as_audio(text):

    response = client.audio.speech.create(
        model = "tts-1",
        voice="alloy",
        input=text
    )

    byte_stream = io.BytesIO(response.content)
    audio = AudioSegment.from_file(byte_stream, format="mp3")

    return audio

In [8]:
stream_text_as_audio("Hello world this is ACM here")